In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib
import matplotlib.pyplot as plt
import scipy
from subprocess import call
from IPython.display import Image
from sklearn.model_selection import train_test_split
from scipy import stats

%matplotlib inline
matplotlib.style.use('bmh')

data_all_thomas = pd.read_excel('../annotations/data_all_thomas.xlsx')
data_labels_video_features = pd.read_csv('../annotations/combined_labels_video_features.csv')
labels = data_labels_video_features[['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val','gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val']]
labels_max = labels[['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val']]
labels_min = labels[['gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val']]
video_features = data_labels_video_features.drop(['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val','gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val'],axis=1)

#Applying z normalization
video_features.apply(stats.zscore)
#Applying L2 normalization
video_features = pd.DataFrame(sk.preprocessing.normalize(video_features, norm='l2',axis=1))

X_train, X_test, y_train, y_test = train_test_split(video_features, labels['gold_gt_min_aro'], train_size=660,test_size=300,shuffle=False)

In [2]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [3]:
import kernel_elm as elm
from sklearn.metrics import classification_report, confusion_matrix,recall_score


elmk = elm.Extreme_Learning_Machine()
beta = elmk.train(X_train,y_train)
kern_test = elmk._kernelize_test(X_train,X_test,'linear')
y_predict = elmk.test(X_test)
recall_score(y_test, y_predict,average='macro')

0.3811590079195713

In [4]:
#np.savetxt('testing_predicted_targets.csv', y_predict, delimiter=',')
np.size(y_predict)

900